# Module 10 Application

## Challenge: Crypto Clustering

In this Challenge, you’ll combine your financial Python programming skills with the new unsupervised learning skills that you acquired in this module.

The CSV file provided for this challenge contains price change data of cryptocurrencies in different periods.

The steps for this challenge are broken out into the following sections:

* Import the Data (provided in the starter code)
* Prepare the Data (provided in the starter code)
* Find the Best Value for `k` Using the Original Data
* Cluster Cryptocurrencies with K-means Using the Original Data
* Optimize Clusters with Principal Component Analysis
* Find the Best Value for `k` Using the PCA Data
* Cluster the Cryptocurrencies with K-means Using the PCA Data
* Visualize and Compare the Results

### Import the Data

This section imports the data into a new DataFrame. It follows these steps:

1. Read  the “crypto_market_data.csv” file from the Resources folder into a DataFrame, and use `index_col="coin_id"` to set the cryptocurrency name as the index. Review the DataFrame.

2. Generate the summary statistics, and use HvPlot to visualize your data to observe what your DataFrame contains.


> **Rewind:** The [Pandas`describe()`function](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.describe.html) generates summary statistics for a DataFrame. 

In [39]:
# Import required libraries and dependencies
import os
import pandas as pd
import hvplot.pandas
from path import Path
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from bokeh.models.tools import HoverTool
from bokeh.models.formatters import NumeralTickFormatter

In [40]:
# Configuration

os.environ['OMP_NUM_THREADS'] = '1'
pd.options.display.float_format = '{:.5f}'.format

In [43]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    Path("Resources/crypto_market_data.csv"),
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [44]:
# Generate summary statistics
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.00000,41.00000,41.00000,41.00000,41.00000,41.00000,41.00000
mean,-0.26969,4.49715,0.18579,1.54569,-0.09412,236.53743,347.66796
std,2.69479,6.37522,8.37694,26.34422,47.36580,435.22530,1247.84288
min,-13.52786,-6.09456,-18.15890,-34.70548,-44.82248,-0.39210,-17.56753
25%,-0.60897,0.04726,-5.02662,-10.43847,-25.90799,21.66042,0.40617
50%,-0.06341,3.29641,0.10974,-0.04237,-7.54455,83.90520,69.69195
75%,0.61209,7.60278,5.51074,4.57813,0.65726,216.17761,168.37251
max,4.84033,20.69459,24.23919,140.79570,223.06437,2227.92782,7852.08970


In [45]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

This section prepares the data before running the K-Means algorithm. It follows these steps:

1. Use the `StandardScaler` module from scikit-learn to normalize the CSV file data. This will require you to utilize the `fit_transform` function.

2. Create a DataFrame that contains the scaled data. Be sure to set the `coin_id` index from the original DataFrame as the index for the new DataFrame. Review the resulting DataFrame.


In [46]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
scaled_data = StandardScaler().fit_transform(df_market_data)

In [47]:
# Create a DataFrame with the scaled data
df_market_data_scaled = pd.DataFrame(
    scaled_data,
    columns=df_market_data.columns
)

# Copy the crypto names from the original data
df_market_data_scaled["coin_id"] = df_market_data.index

# Set the coinid column as index
df_market_data_scaled = df_market_data_scaled.set_index("coin_id")

# Display sample data
df_market_data_scaled.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,0.50853,0.49319,0.77220,0.23546,-0.06750,-0.35595,-0.25164
ethereum,0.18545,0.93445,0.55869,-0.05434,-0.27348,-0.11576,-0.19935
tether,0.02177,-0.70634,-0.02168,-0.06103,0.00800,-0.55025,-0.28206
ripple,-0.04076,-0.81093,0.24946,-0.05039,-0.37316,-0.45826,-0.29555
bitcoin-cash,1.19304,2.00096,1.76061,0.54584,-0.29120,-0.49985,-0.27032


---

### Find the Best Value for k Using the Original Data

In this section, you will use the elbow method to find the best value for `k`.

1. Code the elbow method algorithm to find the best value for `k`. Use a range from 1 to 11. 

2. Plot a line chart with all the inertia values computed with the different values of `k` to visually identify the optimal value for `k`.

3. Answer the following question: What is the best value for `k`?

In [48]:
# Create a list with the number of k-values to try
# Use a range from 1 to 11
k = list(range(1, 11))
k

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [49]:
# Create an empty list to store the inertia values
inertia = []

In [50]:
# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list

for i in k:
    model = KMeans(n_clusters = i, random_state = 0)
    model.fit(df_market_data_scaled)
    inertia.append(model.inertia_)

inertia

C:\Users\Jacob\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:882: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  f"KMeans is known to have a memory leak on Windows "


[287.0,
 195.82021818036043,
 123.19048183836959,
 79.02243535120978,
 65.30237914162501,
 52.88851821977533,
 43.91469044757748,
 37.51703249010357,
 32.48524083218354,
 28.222899290608932]

In [52]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {
    'k': k,
    'inertia': inertia
}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow_data = pd.DataFrame(elbow_data)
df_elbow_data

,k,inertia
0,1,287.00000
1,2,195.82022
2,3,123.19048
3,4,79.02244
4,5,65.30238
5,6,52.88852
6,7,43.91469
7,8,37.51703
8,9,32.48524
9,10,28.22290


In [54]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.

plot_elbow_original = df_elbow_data.hvplot.line(
    title = 'Elbow Data for k Optimization',
    x = 'k',
    y = 'inertia',
    xticks = k,
    width = 1000,
    height = 450
)
plot_elbow_original

:Curve   [k]   (inertia)

#### Answer the following question: What is the best value for k?
**Question:** What is the best value for `k`?

**Answer:** The best number for k is its value where the curve flattens out like an elbow. In the preceding image, this seems to happen at about **k=4**.

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In this section, you will use the K-Means algorithm with the best value for `k` found in the previous section to cluster the cryptocurrencies according to the price changes of cryptocurrencies provided.

1. Initialize the K-Means model with four clusters using the best value for `k`. 

2. Fit the K-Means model using the original data.

3. Predict the clusters to group the cryptocurrencies using the original data. View the resulting array of cluster values.

4. Create a copy of the original data and add a new column with the predicted clusters.

5. Create a scatter plot using hvPlot by setting `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. Color the graph points with the labels found using K-Means and add the crypto name in the `hover_cols` parameter to identify the cryptocurrency represented by each data point.

In [55]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters = 4)

In [56]:
# Fit the K-Means model using the scaled data
model.fit(df_market_data_scaled)

KMeans(n_clusters=4)

In [57]:
# Predict the clusters to group the cryptocurrencies using the scaled data
market_data_clusters = model.predict(df_market_data_scaled)

# View the resulting array of cluster values.
market_data_clusters

array([1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 1, 0, 0, 3, 0, 0, 0, 0])

In [59]:
# Create a copy of the DataFrame
df_market_data_clusters = df_market_data_scaled.copy()

In [60]:
# Add a new column to the DataFrame with the predicted clusters
df_market_data_clusters['Market Data Cluster'] = market_data_clusters

# Display sample data
df_market_data_clusters

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,Market Data Cluster
coin_id,,,,,,,,
bitcoin,0.50853,0.49319,0.77220,0.23546,-0.06750,-0.35595,-0.25164,1
ethereum,0.18545,0.93445,0.55869,-0.05434,-0.27348,-0.11576,-0.19935,1
tether,0.02177,-0.70634,-0.02168,-0.06103,0.00800,-0.55025,-0.28206,0
ripple,-0.04076,-0.81093,0.24946,-0.05039,-0.37316,-0.45826,-0.29555,0
bitcoin-cash,1.19304,2.00096,1.76061,0.54584,-0.29120,-0.49985,-0.27032,1
binancecoin,0.89187,1.32729,0.80021,-0.05715,0.77865,-0.18823,-0.22553,1
chainlink,0.01140,2.57225,1.10165,-0.49050,-0.93195,0.38776,-0.01828,1
cardano,0.10253,1.50800,0.64889,0.32896,-0.48635,0.06508,-0.15543,1
litecoin,0.07750,0.33430,0.85852,-0.01265,-0.36648,-0.48627,-0.29235,1


In [61]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.

# Format for the axis ticks. Add to plot 'yformatter' option
formatter = NumeralTickFormatter(format=('0.0'))

# Format the labels and number format for hovering tooltips. Add to plot 'tools' option
hover = HoverTool()
hover.tooltips =[
    ('Coin Name:','@coin_id'), 
    ('24H Price Change:', '@price_change_percentage_24h{0.00}%'),
    (' 7D Price Change:', '@price_change_percentage_7d{0.00}%')   
]

df_market_data_clusters.hvplot.scatter(
    title =  'Cryptocurrency Clusters: Price Change, 24h vs 7d',
    x = 'price_change_percentage_24h',
    xlabel = '24H Price Change (%)',
    y = 'price_change_percentage_7d',
    ylabel = '7D Price Change (%)',
    by = 'Market Data Cluster',
    hover_cols = ['coin_id', 'PC1', 'PC2', 'PC3'],
    tools = [hover],
    yformatter = '%.0f',
    xformatter = '%.0f',
    width = 1000,
    height = 450,
    fontsize = {'title': '20pt', 'xlabel': '15pt', 'ylabel': '15pt', 'ticks': '10pt'},
    grid = True
)

:NdOverlay   [Market Data Cluster]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

### Optimize Clusters with Principal Component Analysis

In this section, you will perform a principal component analysis (PCA) and reduce the features to three principal components.

1. Create a PCA model instance and set `n_components=3`.

2. Use the PCA model to reduce to three principal components. View the first five rows of the DataFrame. 

3. Retrieve the explained variance to determine how much information can be attributed to each principal component.

4. Answer the following question: What is the total explained variance of the three principal components?

5. Create a new DataFrame with the PCA data. Be sure to set the `coin_id` index from the original DataFrame as the index for the new DataFrame. Review the resulting DataFrame.

In [62]:
# Create a PCA model instance and set `n_components=3`.
model = PCA(n_components=3)

In [63]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.
market_data_scaled_pca = model.fit_transform(df_market_data_scaled)

# View the first five rows of the DataFrame. 
market_data_scaled_pca[0:5]

array([[-0.60066733,  0.84276006,  0.46159457],
       [-0.45826071,  0.45846566,  0.95287678],
       [-0.43306981, -0.16812638, -0.64175193],
       [-0.47183495, -0.22266008, -0.47905316],
       [-1.15779997,  2.04120919,  1.85971527]])

In [64]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.

print(f'Explained variance ratio: {model.explained_variance_ratio_} explaining {model.explained_variance_ratio_.sum():0.2%} total variance.')

Explained variance ratio: [0.3719856  0.34700813 0.17603793] explaining 89.50% total variance.


#### Answer the following question: What is the total explained variance of the three principal components?

**Question:** What is the total explained variance of the three principal components?

**Answer:** The three principal components account for **89.50%** of the explained variance.

In [65]:
# Create a new DataFrame with the PCA data.

# Creating a DataFrame with the PCA data
df_market_data_scaled_pca = pd.DataFrame(market_data_scaled_pca, columns = ['PC1','PC2','PC3'])

# Copy the crypto names from the original data
df_market_data_scaled_pca['coin_id'] = df_market_data_scaled.index.str.capitalize()

# Set the coinid column as index
df_market_data_scaled_pca.set_index('coin_id', inplace = True)

# Display sample data
df_market_data_scaled_pca.head()

,PC1,PC2,PC3
coin_id,,,
Bitcoin,-0.60067,0.84276,0.46159
Ethereum,-0.45826,0.45847,0.95288
Tether,-0.43307,-0.16813,-0.64175
Ripple,-0.47183,-0.22266,-0.47905
Bitcoin-cash,-1.15780,2.04121,1.85972


---

### Find the Best Value for k Using the PCA Data

In this section, you will use the elbow method to find the best value for `k` using the PCA data.

1. Code the elbow method algorithm and use the PCA data to find the best value for `k`. Use a range from 1 to 11. 

2. Plot a line chart with all the inertia values computed with the different values of `k` to visually identify the optimal value for `k`.

3. Answer the following questions: What is the best value for k when using the PCA data? Does it differ from the best k value found using the original data?

In [66]:
# Create a list with the number of k-values to try
# Use a range from 1 to 11
k = list(range(1,11))
k

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [67]:
# Create an empy list to store the inertia values
inertia = []

In [68]:
# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list

for i in k:
    model = KMeans(n_clusters=i)
    model.fit(df_market_data_scaled_pca)
    inertia.append(model.inertia_)

inertia

C:\Users\Jacob\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:882: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  f"KMeans is known to have a memory leak on Windows "


[256.87408556789256,
 165.90199402036015,
 93.77462568057294,
 49.665496651797355,
 38.072792150143194,
 27.720626194899804,
 21.145312121622496,
 17.28546035051391,
 13.74279196048056,
 10.57237979909294]

In [69]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {
    'k':k,
    'inertia':inertia
}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow_data = pd.DataFrame(elbow_data)
df_elbow_data

,k,inertia
0,1,256.87409
1,2,165.90199
2,3,93.77463
3,4,49.66550
4,5,38.07279
5,6,27.72063
6,7,21.14531
7,8,17.28546
8,9,13.74279
9,10,10.57238


In [70]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.

df_elbow_data.hvplot.line(
    title = 'Elbow Data for k Optimization',
    x = 'k',
    y = 'inertia',
    xticks = k,
    width = 1000,
    height = 450)

:Curve   [k]   (inertia)

#### Answer the following questions: What is the best value for k when using the PCA data? Does it differ from the best k value found using the original data?
* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** The best number for k is its value where the curve flattens out like an elbow. In the preceding image, this seems to happen at about **k=4**.


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** No

---

### Cluster Cryptocurrencies with K-means Using the PCA Data

In this section, you will use the PCA data and the K-Means algorithm with the best value for `k` found in the previous section to cluster the cryptocurrencies according to the principal components.

1. Initialize the K-Means model with four clusters using the best value for `k`. 

2. Fit the K-Means model using the PCA data.

3. Predict the clusters to group the cryptocurrencies using the PCA data. View the resulting array of cluster values.

4. Add a new column to the DataFrame with the PCA data to store the predicted clusters.

5. Create a scatter plot using hvPlot by setting `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. Color the graph points with the labels found using K-Means and add the crypto name in the `hover_cols` parameter to identify the cryptocurrency represented by each data point.

In [71]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters = 4)

In [72]:
# Fit the K-Means model using the PCA data
model.fit(df_market_data_scaled_pca)

KMeans(n_clusters=4)

In [73]:
# Predict the clusters to group the cryptocurrencies using the PCA data
market_data_clusters_pca = model.predict(df_market_data_scaled_pca)

# View the resulting array of cluster values.
market_data_clusters_pca

array([1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 3, 1, 0, 0, 2, 0, 0, 0, 0])

In [75]:
# Create a copy of the DataFrame with the PCA data
df_market_data_clusters_pca = df_market_data_scaled.copy()

# Capitalize coin name for more effective visualizations
df_market_data_clusters_pca.index = df_market_data_clusters.index.str.capitalize()

# Add a new column to the DataFrame with the predicted clusters
df_market_data_clusters_pca['Market Data Cluster'] = market_data_clusters_pca

# Display sample data
df_market_data_clusters_pca.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,Market Data Cluster
coin_id,,,,,,,,
Bitcoin,0.50853,0.49319,0.77220,0.23546,-0.06750,-0.35595,-0.25164,1
Ethereum,0.18545,0.93445,0.55869,-0.05434,-0.27348,-0.11576,-0.19935,1
Tether,0.02177,-0.70634,-0.02168,-0.06103,0.00800,-0.55025,-0.28206,0
Ripple,-0.04076,-0.81093,0.24946,-0.05039,-0.37316,-0.45826,-0.29555,0
Bitcoin-cash,1.19304,2.00096,1.76061,0.54584,-0.29120,-0.49985,-0.27032,1


In [76]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.

# Format for the axis ticks. Add to plot 'yformatter' option
formatter = NumeralTickFormatter(format=('0.0'))

# Format the labels and number format for hovering tooltips. Add to plot 'tools' option
hover = HoverTool()
hover.tooltips =[
    ('Coin Name:','@coin_id'), 
    ('24H Price Change:', '@price_change_percentage_24h{0.00}%'),
    (' 7D Price Change:', '@price_change_percentage_7d{0.00}%')   
]

df_market_data_clusters_pca.hvplot.scatter(
    title =  'Cryptocurrency Clusters: Price Change, 24h vs 7d [PCA]',
    x = 'price_change_percentage_24h',
    xlabel = '24H Price Change (%)',
    y = 'price_change_percentage_7d',
    ylabel = '7D Price Change (%)',
    by = 'Market Data Cluster',
    hover_cols = ['coin_id', 'PC1', 'PC2', 'PC3'],
    tools = [hover],
    yformatter = '%.0f',
    xformatter = '%.0f',
    width = 1000,
    height = 450,
    fontsize = {'title': '20pt', 'xlabel': '15pt', 'ylabel': '15pt', 'ticks': '10pt'},
    grid = True
)

:NdOverlay   [Market Data Cluster]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

---

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

1. Create a composite plot using hvPlot and the plus (`+`) operator to contrast the Elbow Curve that you created to find the best value for `k` with the original and the PCA data.

2. Create a composite plot using hvPlot and the plus (`+`) operator to contrast the cryptocurrencies clusters using the original and the PCA data.

3. Answer the following question: After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

> **Rewind:** Back in Lesson 3 of Module 6, you learned how to create composite plots. You can look at that lesson to review how to make these plots; also, you can check [the hvPlot documentation](https://holoviz.org/tutorial/Composing_Plots.html).

In [38]:
# Composite plot to contrast the Elbow curves
# YOUR CODE HERE!

In [39]:
# Compoosite plot to contrast the clusters
# YOUR CODE HERE!

#### Answer the following question: After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** # YOUR ANSWER HERE!